In [ ]:
%pylab inline

In [ ]:
def generate_perlin_noise_2d(shape, res):
    """
    Stolen from: https://pvigier.github.io/2018/06/13/perlin-noise-numpy.html
    """
    def f(t):
        return 6*t**5 - 15*t**4 + 10*t**3
    
    delta = (res[0] / shape[0], res[1] / shape[1])
    d = (shape[0] // res[0], shape[1] // res[1])
    grid = np.mgrid[0:res[0]:delta[0],0:res[1]:delta[1]].transpose(1, 2, 0) % 1
    # Gradients
    angles = 2*np.pi*np.random.rand(res[0]+1, res[1]+1)
    gradients = np.dstack((np.cos(angles), np.sin(angles)))
    g00 = gradients[0:-1,0:-1].repeat(d[0], 0).repeat(d[1], 1)
    g10 = gradients[1:,0:-1].repeat(d[0], 0).repeat(d[1], 1)
    g01 = gradients[0:-1,1:].repeat(d[0], 0).repeat(d[1], 1)
    g11 = gradients[1:,1:].repeat(d[0], 0).repeat(d[1], 1)
    # Ramps
    n00 = np.sum(grid * g00, 2)
    n10 = np.sum(np.dstack((grid[:,:,0]-1, grid[:,:,1])) * g10, 2)
    n01 = np.sum(np.dstack((grid[:,:,0], grid[:,:,1]-1)) * g01, 2)
    n11 = np.sum(np.dstack((grid[:,:,0]-1, grid[:,:,1]-1)) * g11, 2)
    # Interpolation
    t = f(grid)
    n0 = n00*(1-t[:,:,0]) + t[:,:,0]*n10
    n1 = n01*(1-t[:,:,0]) + t[:,:,0]*n11
    return np.sqrt(2)*((1-t[:,:,1])*n0 + t[:,:,1]*n1)

def split_2d(array, splits):
    x, y = splits
    return np.split(np.concatenate(np.split(array, y, axis=1)), x*y)

In [ ]:
m = generate_perlin_noise_2d((1500, 1500), (300, 300))
chunks = split_2d(m, (30, 30))
imshow(chunks[0] > 0.3)

In [ ]:
def _n2_arch_map(tile):
    if tile > 0.3:
        return np.array(["caveblackrock"])
    else:
        return np.array(["blackrock"])
    
def sky_arch_map(tile):
    if tile > 0.3:
        return ["empty_sky", "cloudfloor_bl"]
    else:
        return ["empty_sky"]

In [ ]:
def to_map(m, name):
    h, w = m.shape
    lines = []
    lines.append("arch map")
    lines.append("name %s" % name)
    lines.append("width %d" % w)
    lines.append("height %d" % h)
    #lines.append("darkness %d" % 2)
    lines.append("outdoor 1")
    lines.append("tile_path_1 nonexistent")
    lines.append("end")
    for x in range(w):
        for y in range(h):
            for arch in arch_map(m[x,y]):
                lines.append("arch %s" % arch)
                lines.append("x %d" % x)
                lines.append("y %d" % y)
                lines.append("elevation %d" % -2000)
                lines.append("end")
    return lines

def write_map(m, name):
    with open(name, 'w') as f:
        f.write("\n".join(to_map(m, name)))
        
def write_world(name, chunks, h, w, start=100):
    for n, c in enumerate(chunks):
        x = n % w + start
        y = n // h + start
        write_map(c, "%s_%d_%d" % (name, x, y))

In [ ]:
arch_map = _n2_arch_map
write_world("bottomworld_-2", chunks[:1], 30, 30)

In [ ]:
arch_map = sky_arch_map
write_world("bottomworld_1", chunks, 30, 30)

In [ ]:
arch_map = sky_arch_map
write_world("bottomworld_2", chunks, 30, 30)